In [ ]:
pip install torch torchvision kagglehub matplotlib seaborn tqdm

In [ ]:
import os
import copy
import time
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from torch import nn, optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve
from tqdm import tqdm
import kagglehub

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Download dataset
path = kagglehub.dataset_download("manjilkarki/deepfake-and-real-images")
dataset_dir = os.path.join(path, "Dataset")


In [ ]:
# Define transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ]),
}

In [ ]:
# Load datasets
train_dataset = datasets.ImageFolder(os.path.join(dataset_dir, "Train"), data_transforms['train'])
val_dataset = datasets.ImageFolder(os.path.join(dataset_dir, "Validation"), data_transforms['val'])
test_dataset = datasets.ImageFolder(os.path.join(dataset_dir, "Test"), data_transforms['test'])

In [ ]:
# Create dataloaders
batch_size = 32
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4),
    'val': DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4),
    'test': DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
}

In [ ]:
import matplotlib.pyplot as plt

def plot_dataset_insights(dataset, split_name):
    # Calculate class counts
    class_counts = {cls: 0 for cls in dataset.classes}
    for _, label in dataset.samples:
        class_counts[dataset.classes[label]] += 1

    # Prepare data for plotting
    classes = list(class_counts.keys())
    counts = list(class_counts.values())

    # Create bar plot
    plt.figure(figsize=(6, 4))
    bars = plt.bar(classes, counts, color=['skyblue', 'salmon'])
    plt.title(f"{split_name} Dataset Insights")
    plt.xlabel("Class")
    plt.ylabel("Number of images")

    # Add text labels on top of each bar
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.05*yval, f'{yval}', ha='center', va='bottom')

    plt.show()

# Plot insights for each dataset split
plot_dataset_insights(train_dataset, "Train")
plot_dataset_insights(val_dataset, "Validation")
plot_dataset_insights(test_dataset, "Test")

In [ ]:
# Initialize ResNet50
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)
model = model.to(device)

In [ ]:
# Training setup
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, min_lr=1e-7)

In [ ]:
# Configuration for training loop
num_epochs = 20
early_stop_patience = 3
best_auc = 0

In [ ]:
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()
            
        running_loss = 0.0
        correct = 0
        total = 0
        all_labels = []
        all_preds = []
        
        # Wrap dataloader with tqdm for batch-level progress
        pbar = tqdm(dataloaders[phase], desc=f"{phase.capitalize()} Phase", leave=False)
        for inputs, labels in pbar:
            inputs = inputs.to(device)
            labels = labels.to(device).float().unsqueeze(1)
            
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()
                
            running_loss += loss.item() * inputs.size(0)
            preds = torch.sigmoid(outputs).detach().cpu().numpy()
            batch_preds = (preds >= 0.5).astype(int)
            batch_labels = labels.cpu().numpy().astype(int)
            
            all_preds.extend(preds.flatten())
            all_labels.extend(batch_labels.flatten())
            correct += (batch_preds == batch_labels).sum()
            total += inputs.size(0)
            
            pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "Acc": f"{(batch_preds == batch_labels).mean():.4f}"
            })
        
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = correct / total
        epoch_auc = roc_auc_score(all_labels, all_preds)
        current_lr = optimizer.param_groups[0]['lr']
        
        print(f"{phase.capitalize()} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}, AUC: {epoch_auc:.4f}, LR: {current_lr:.6f}")

        if phase == 'val':
            scheduler.step(epoch_auc)
            if epoch_auc > best_auc:
                best_auc = epoch_auc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), 'best_model.pth')
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

    if epochs_no_improve >= early_stop_patience:
        print("Early stopping triggered")
        break

In [ ]:
torch.save(model, "final_model_resnet50.pth");

In [ ]:
# Testing phase with detailed metrics
model.eval()
test_loss = 0.0
correct = 0
total = 0
all_labels = []
all_preds = []

pbar_test = tqdm(dataloaders['test'], desc="Testing Phase", leave=False)
with torch.no_grad():
    for inputs, labels in pbar_test:
        inputs = inputs.to(device)
        labels = labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        test_loss += loss.item() * inputs.size(0)
        preds = torch.sigmoid(outputs).cpu().numpy()
        batch_preds = (preds >= 0.5).astype(int)
        batch_labels = labels.cpu().numpy().astype(int)
        
        all_preds.extend(preds.flatten())
        all_labels.extend(batch_labels.flatten())
        correct += (batch_preds == batch_labels).sum()
        total += inputs.size(0)
        
        pbar_test.set_postfix({
            "Loss": f"{loss.item():.4f}",
            "Acc": f"{(batch_preds == batch_labels).mean():.4f}"
        })

test_loss = test_loss / len(dataloaders['test'].dataset)
test_acc = correct / total
test_auc = roc_auc_score(all_labels, all_preds)
current_lr = optimizer.param_groups[0]['lr']

print("\nTest Metrics:")
print(f"Loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}, AUC: {test_auc:.4f}, LR: {current_lr:.6f}")

In [ ]:
# Confusion Matrix
cm = confusion_matrix(all_labels, (np.array(all_preds) >= 0.5))
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Fake', 'Real'], yticklabels=['Fake', 'Real'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# ROC Curve
fpr, tpr, _ = roc_curve(all_labels, all_preds)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC = {test_auc:.2f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

In [ ]:
model.load_state_dict(best_model_wts)

torch.save(model, "best_model_resnet50.pth")

In [ ]:
# Testing for best_model
model.eval()
test_loss = 0.0
correct = 0
total = 0
all_labels = []
all_preds = []

pbar_test = tqdm(dataloaders['test'], desc="Testing Phase", leave=False)
with torch.no_grad():
    for inputs, labels in pbar_test:
        inputs = inputs.to(device)
        labels = labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        test_loss += loss.item() * inputs.size(0)
        preds = torch.sigmoid(outputs).cpu().numpy()
        batch_preds = (preds >= 0.5).astype(int)
        batch_labels = labels.cpu().numpy().astype(int)
        
        all_preds.extend(preds.flatten())
        all_labels.extend(batch_labels.flatten())
        correct += (batch_preds == batch_labels).sum()
        total += inputs.size(0)
        
        pbar_test.set_postfix({
            "Loss": f"{loss.item():.4f}",
            "Acc": f"{(batch_preds == batch_labels).mean():.4f}"
        })

test_loss = test_loss / len(dataloaders['test'].dataset)
test_acc = correct / total
test_auc = roc_auc_score(all_labels, all_preds)
current_lr = optimizer.param_groups[0]['lr']

print("\nTest Metrics:")
print(f"Loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}, AUC: {test_auc:.4f}, LR: {current_lr:.6f}")

In [ ]:
# Confusion Matrix
cm = confusion_matrix(all_labels, (np.array(all_preds) >= 0.5))
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# ROC Curve
fpr, tpr, _ = roc_curve(all_labels, all_preds)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC = {test_auc:.2f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

precision, recall, thresholds = precision_recall_curve(all_labels, all_preds)
ap = average_precision_score(all_labels, all_preds)

plt.figure(figsize=(6, 4))
plt.step(recall, precision, where='post', label=f'AP = {ap:.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(all_preds, bins=20, alpha=0.7, color='gray')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.title('Histogram of Predicted Probabilities')
plt.show()

In [ ]:
from sklearn.calibration import calibration_curve

prob_true, prob_pred = calibration_curve(all_labels, all_preds, n_bins=10)

plt.figure(figsize=(6, 4))
plt.plot(prob_pred, prob_true, marker='o', linewidth=1, label='Calibration curve')
plt.plot([0, 1], [0, 1], linestyle='--', label='Perfect calibration')
plt.xlabel('Mean predicted probability')
plt.ylabel('Fraction of positives')
plt.title('Calibration Plot')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import f1_score

# Compute F1 score for a range of thresholds
thresholds = np.linspace(0, 1, 100)
f1_scores = [f1_score(all_labels, (np.array(all_preds) >= t).astype(int)) for t in thresholds]

plt.figure(figsize=(6, 4))
plt.plot(thresholds, f1_scores, marker='o')
plt.xlabel("Threshold")
plt.ylabel("F1 Score")
plt.title("F1 Score vs Threshold")
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd

# Create a DataFrame with true labels and predicted probabilities
df = pd.DataFrame({"label": all_labels, "pred": all_preds})
df = df.sort_values("pred", ascending=False).reset_index(drop=True)
df["cumulative_positive"] = df["label"].cumsum()
df["percentage_positive"] = df["cumulative_positive"] / df["label"].sum()
df["percentage_data"] = (df.index + 1) / len(df)

plt.figure(figsize=(6, 4))
plt.plot(df["percentage_data"], df["percentage_positive"], label="Model")
plt.plot([0, 1], [0, 1], linestyle="--", label="Random")
plt.xlabel("Percentage of Data")
plt.ylabel("Percentage of Positives Captured")
plt.title("Cumulative Gains Chart")
plt.legend()
plt.grid(True)
plt.show()